<a href="https://colab.research.google.com/github/shivangsingh26/FL-BC-BTP/blob/master/FedWPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## FEDWPR (5% WITHOUT HYPERPARAMETER TUNING)

In [ ]:
!pip install tensorflow federated
!pip install gdown

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import zipfile

# Unzip the datasets
zip_path = '/content/drive/My Drive/BTP.zip'
unzip_path = '/content/datasets'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)


Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Function to load and preprocess data with augmentation
def load_data(client_path):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        os.path.join(client_path, 'train'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        os.path.join(client_path, 'test'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    return train_generator, test_generator

clients = ["/content/datasets/Non-iid datasets/non_iid_subset_1","/content/datasets/Non-iid datasets/non_iid_subset_2","/content/datasets/Non-iid datasets/non_iid_subset_3","/content/datasets/Non-iid datasets/non_iid_subset_4",]
data_paths = [os.path.join(unzip_path, client) for client in clients]

train_generators = []
test_generators = []
for path in data_paths:
    train_gen, test_gen = load_data(path)
    train_generators.append(train_gen)
    test_generators.append(test_gen)


Found 19012 images belonging to 19 classes.
Found 4748 images belonging to 19 classes.
Found 19185 images belonging to 19 classes.
Found 4791 images belonging to 19 classes.
Found 19703 images belonging to 19 classes.
Found 4921 images belonging to 19 classes.
Found 19530 images belonging to 19 classes.
Found 4878 images belonging to 19 classes.


In [ ]:
def create_model(num_classes=19):
    base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                                   include_top=False,
                                                   weights='imagenet')
    base_model.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [ ]:
import numpy as np

# Define number of communication rounds and hyperparameters grid
num_rounds = 5
hyperparameters_grid = {
    'RR': [0.3, 0.5, 0.7],
    'epochs': [5, 10]
}

def fed_wpr(client_models, RR):
    new_weights_list = []
    client_weights = [model.get_weights() for model in client_models]

    for client_id in range(len(client_models)):
        new_weights = []
        for layer_weights in zip(*client_weights):
            weighted_sum = np.zeros_like(layer_weights[0])
            for j in range(len(client_models)):
                weighted_sum += RR * layer_weights[j]
            personalized_weights = (1 - RR) * layer_weights[client_id] + weighted_sum
            new_weights.append(personalized_weights)
        new_weights_list.append(new_weights)

    return new_weights_list

def train_and_evaluate(client_models, train_generators, test_generators, RR, epochs, num_rounds=10):
    for round_num in range(num_rounds):
        print(f'Round {round_num+1}/{num_rounds}')

        for i in range(4):
            print(f'Training client {i+1}')
            client_models[i].fit(train_generators[i], epochs=epochs, validation_data=test_generators[i])

        new_weights_list = fed_wpr(client_models, RR)

        for i in range(4):
            client_models[i].set_weights(new_weights_list[i])

    avg_accuracy = 0
    for i in range(4):
        loss, accuracy = client_models[i].evaluate(test_generators[i])
        avg_accuracy += accuracy
        print(f'Client {i+1} - Loss: {loss}, Accuracy: {accuracy}')

    avg_accuracy /= 4
    return avg_accuracy

def grid_search(hyperparameters_grid):
    best_accuracy = 0
    best_params = {}

    for RR in hyperparameters_grid['RR']:
        for epochs in hyperparameters_grid['epochs']:
            print(f'Evaluating RR = {RR}, epochs = {epochs}')
            client_models = [create_model(num_classes=19) for _ in range(4)]

            avg_accuracy = train_and_evaluate(client_models, train_generators, test_generators, RR, epochs)

            print(f'Average accuracy for RR = {RR}, epochs = {epochs}: {avg_accuracy}')

            if avg_accuracy > best_accuracy:
                best_accuracy = avg_accuracy
                best_params = {'RR': RR, 'epochs': epochs}

    return best_params, best_accuracy

best_params, best_accuracy = grid_search(hyperparameters_grid)
print(f'Best parameters: {best_params} with average accuracy: {best_accuracy}')


Evaluating RR = 0.3, epochs = 5
9406464/9406464 [==============================] - 2s 0us/step
Round 1/10
Training client 1
Epoch 1/5
595/595 [==============================] - 322s 526ms/step - loss: 2.5215 - accuracy: 0.2138 - val_loss: 2.2394 - val_accuracy: 0.3317
Epoch 2/5
595/595 [==============================] - 266s 446ms/step - loss: 2.2101 - accuracy: 0.3029 - val_loss: 1.8453 - val_accuracy: 0.4164
Epoch 3/5
595/595 [==============================] - 263s 442ms/step - loss: 2.1064 - accuracy: 0.3348 - val_loss: 1.6908 - val_accuracy: 0.4558
Epoch 4/5
595/595 [==============================] - 262s 440ms/step - loss: 2.0058 - accuracy: 0.3617 - val_loss: 1.5987 - val_accuracy: 0.5065
Epoch 5/5
595/595 [==============================] - 265s 445ms/step - loss: 1.9645 - accuracy: 0.3811 - val_loss: 1.6173 - val_accuracy: 0.4941
Training client 2
Epoch 1/5
600/600 [==============================] - 328s 539ms/step - loss: 2.3289 - accuracy: 0.2792 - val_loss: 1.6640 - val_accur

In [ ]:
for i in range(4):
    loss, accuracy = client_models[i].evaluate(test_generators[i])
    print(f'Client {i+1} - Loss: {loss}, Accuracy: {accuracy}')

## FEDWPR (5% WITH HYPERPARAMETER TUNING)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import zipfile

# Unzip the datasets
zip_path = '/content/drive/My Drive/BTP.zip'
unzip_path = '/content/datasets'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import ParameterGrid

# Function to load and preprocess data with augmentation
def load_data(client_path):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        os.path.join(client_path, 'train'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        os.path.join(client_path, 'test'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    return train_generator, test_generator

clients = ["/content/datasets/Non-iid datasets/non_iid_subset_1","/content/datasets/Non-iid datasets/non_iid_subset_2","/content/datasets/Non-iid datasets/non_iid_subset_3","/content/datasets/Non-iid datasets/non_iid_subset_4",]
data_paths = [os.path.join(unzip_path, client) for client in clients]

train_generators = []
test_generators = []
for path in data_paths:
    train_gen, test_gen = load_data(path)
    train_generators.append(train_gen)
    test_generators.append(test_gen)

def create_model(num_classes=19):
    base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                                   include_top=False,
                                                   weights='imagenet')
    base_model.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Define the FedWPR aggregation function
def fed_wpr(client_models, RR):
    new_weights_list = []
    client_weights = [model.get_weights() for model in client_models]

    for client_id in range(len(client_models)):
        new_weights = []
        for layer_weights in zip(*client_weights):
            weighted_sum = np.sum([w * RR for w in layer_weights], axis=0)
            personalized_weights = (1 - RR) * layer_weights[client_id] + weighted_sum
            new_weights.append(personalized_weights)
        new_weights_list.append(new_weights)

    return new_weights_list

# Train and evaluate function with learning rate scheduling and early stopping
def train_and_evaluate(client_models, train_generators, test_generators, RR, epochs, num_rounds=10):
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

    for round_num in range(num_rounds):
        print(f'Round {round_num+1}/{num_rounds}')

        for i in range(4):
            print(f'Training client {i+1}')
            client_models[i].fit(train_generators[i], epochs=epochs, validation_data=test_generators[i],
                                 callbacks=[callback, lr_scheduler])

        new_weights_list = fed_wpr(client_models, RR)

        for i in range(4):
            client_models[i].set_weights(new_weights_list[i])

    avg_accuracy = 0
    for i in range(4):
        loss, accuracy = client_models[i].evaluate(test_generators[i])
        avg_accuracy += accuracy
        print(f'Client {i+1} - Loss: {loss}, Accuracy: {accuracy}')

    avg_accuracy /= 4
    return avg_accuracy

# Custom grid search function with detailed monitoring
def grid_search(hyperparameters_grid):
    best_accuracy = 0
    best_params = {}

    param_grid = list(ParameterGrid(hyperparameters_grid))

    for params in param_grid:
        RR = params['RR']
        epochs = params['epochs']

        print(f'Evaluating RR = {RR}, epochs = {epochs}')
        client_models = [create_model(num_classes=19) for _ in range(4)]

        avg_accuracy = train_and_evaluate(client_models, train_generators, test_generators, RR, epochs)

        print(f'Average accuracy for RR = {RR}, epochs = {epochs}: {avg_accuracy}')

        if avg_accuracy > best_accuracy:
            best_accuracy = avg_accuracy
            best_params = {'RR': RR, 'epochs': epochs}

    return best_params, best_accuracy

# Define hyperparameters grid
hyperparameters_grid = {
    'RR': [0.3, 0.5, 0.7],
    'epochs': [10, 20]
}

best_params, best_accuracy = grid_search(hyperparameters_grid)
print(f'Best parameters: {best_params} with average accuracy: {best_accuracy}')


Found 19012 images belonging to 19 classes.
Found 4748 images belonging to 19 classes.
Found 19185 images belonging to 19 classes.
Found 4791 images belonging to 19 classes.
Found 19703 images belonging to 19 classes.
Found 4921 images belonging to 19 classes.
Found 19530 images belonging to 19 classes.
Found 4878 images belonging to 19 classes.
Evaluating RR = 0.3, epochs = 10
9406464/9406464 [==============================] - 1s 0us/step
Round 1/10
Training client 1
Epoch 1/10
595/595 [==============================] - 347s 569ms/step - loss: 2.5186 - accuracy: 0.2171 - val_loss: 2.1193 - val_accuracy: 0.3370 - lr: 0.0010
Epoch 2/10
595/595 [==============================] - 300s 504ms/step - loss: 2.2344 - accuracy: 0.2940 - val_loss: 1.8850 - val_accuracy: 0.4021 - lr: 0.0010
Epoch 3/10
595/595 [==============================] - 274s 460ms/step - loss: 2.1054 - accuracy: 0.3365 - val_loss: 1.8559 - val_accuracy: 0.4185 - lr: 0.0010
Epoch 4/10
595/595 [==============================

## FEDWPR(NEW)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install necessary libraries
# !pip install tensorflow
!pip install tensorflow_federated


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.6/721.6 kB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.9/230.9 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Extract the dataset
zip_path = '/content/drive/MyDrive/BTP.zip'
extract_path = '/content/finger_vein_dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

KeyboardInterrupt: 

In [ ]:
# Create data loaders for each client
data_dir = extract_path
batch_size = 32
img_height = 224
img_width = 224

def create_data_loader(data_path, img_height, img_width, batch_size):
    datagen = ImageDataGenerator(rescale=1./255)
    data = datagen.flow_from_directory(
        data_path,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

    return data

clients_train_data = []
clients_test_data = []
for i in range(1, 5):
    client_train_path = os.path.join(data_dir, f'/content/finger_vein_dataset/Non-iid datasets/non_iid_subset_{i}', 'train')
    client_test_path = os.path.join(data_dir, f'/content/finger_vein_dataset/Non-iid datasets/non_iid_subset_{i}', 'test')
    train_data = create_data_loader(client_train_path, img_height, img_width, batch_size)
    test_data = create_data_loader(client_test_path, img_height, img_width, batch_size)
    clients_train_data.append(train_data)
    clients_test_data.append(test_data)

Found 19012 images belonging to 19 classes.
Found 4748 images belonging to 19 classes.
Found 19185 images belonging to 19 classes.
Found 4791 images belonging to 19 classes.
Found 19703 images belonging to 19 classes.
Found 4921 images belonging to 19 classes.
Found 19530 images belonging to 19 classes.
Found 4878 images belonging to 19 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

def create_model(num_classes):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model


In [ ]:
import numpy as np
from tensorflow.keras.optimizers import Adam

# Client training function
def client_train(model, train_data, epochs=1):
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_data, epochs=epochs, verbose=1)
    return model.get_weights()

# FedWPR aggregation function
def fedwpr_aggregation(local_weights, RR):
    N = len(local_weights)
    new_weights = []

    for layer_weights in zip(*local_weights):
        new_layer_weights = np.zeros_like(layer_weights[0])
        for i in range(N):
            new_layer_weights += RR * np.array(layer_weights[i]) / N
        for i in range(N):
            new_layer_weights += (1 - RR) * np.array(layer_weights[i]) / N
        new_weights.append(new_layer_weights)

    return new_weights

def fedwpr_aggregation(local_weights, RR):
    N = len(local_weights)
    aggregated_weights = []

    for layer_weights in zip(*local_weights):
        rr_component = np.zeros_like(layer_weights[0])
        one_minus_rr_component = np.zeros_like(layer_weights[0])

        # Calculate RR component
        for weights in layer_weights:
            rr_component += RR * np.array(weights) / N

        # Calculate (1 - RR) component
        for weights in layer_weights:
            one_minus_rr_component += (1 - RR) * np.array(weights) / N

        # Combine both components
        new_layer_weights = rr_component + one_minus_rr_component
        aggregated_weights.append(new_layer_weights)

    return aggregated_weights

# Simulate federated learning with FedFV and FedWPR
global_model = create_model(num_classes=19)  # Assuming 20 classes for the global model
num_rounds = 10
RR = 0.5  # Set the RR value as needed

for round_num in range(num_rounds):
    local_weights = []
    for client_num, train_data in enumerate(clients_train_data):
        client_model = create_model(num_classes=train_data.num_classes)
        client_model.set_weights(global_model.get_weights())
        client_weights = client_train(client_model, train_data, epochs=1)
        local_weights.append(client_weights)

    # Apply FedWPR aggregation
    new_global_weights = fedwpr_aggregation(local_weights, RR)
    global_model.set_weights(new_global_weights)

    # Distribute the aggregated model back to clients
    for client_num, train_data in enumerate(clients_train_data):
        client_model.set_weights(new_global_weights)

    print(f'Round {round_num + 1} completed')

# Save the global model
global_model.save('/content/drive/MyDrive/federated_global_model_fedwpr.h5')


533/611 [=========================>....] - ETA: 1:21 - loss: 0.7660 - accuracy: 0.7630

In [ ]:
def evaluate_model(model, test_data):
    loss, accuracy = model.evaluate(test_data, verbose=0)
    return loss, accuracy

# Evaluate the global model on each client's test data
for client_num, test_data in enumerate(clients_test_data):
    loss, accuracy = evaluate_model(global_model, test_data)
    print(f'Client {client_num + 1} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')

In [ ]:
import os
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.data import Dataset

print("TensorFlow version:", tf.__version__)
print("OpenCV version:", cv2.__version__)

# Set the paths
data_dir = 'Non-iid datasets'
batch_size = 32
img_height = 224
img_width = 224
num_classes = 19  # Number of classes

def load_and_preprocess_image(path, label):
    img = cv2.imread(path.decode('utf-8'))  # path needs to be decoded from bytes
    img = cv2.resize(img, (img_width, img_height))
    img = img / 255.0  # Normalize to [0, 1]
    label = tf.one_hot(label, num_classes)  # One-hot encode the label
    return img.astype(np.float32), label.numpy().astype(np.float32)

def create_dataset(data_dir, batch_size):
    image_paths = []
    labels = []
    class_names = os.listdir(data_dir)

    for label, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            image_paths.append(img_path)
            labels.append(label)

    image_paths = np.array(image_paths)
    labels = np.array(labels)

    dataset = Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(lambda x, y: tf.numpy_function(load_and_preprocess_image, [x, y], [tf.float32, tf.float32]))

    # Explicitly set the shapes
    dataset = dataset.map(lambda x, y: (tf.ensure_shape(x, [img_height, img_width, 3]), tf.ensure_shape(y, [num_classes])))

    dataset = dataset.shuffle(buffer_size=len(image_paths))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset, len(class_names)

clients_train_data = []
clients_test_data = []

for i in range(1, 5):  # Adjusted to 4 clients
    client_train_path = os.path.join(data_dir, f'client_{i}', 'train')
    client_test_path = os.path.join(data_dir, f'client_{i}', 'test')
    train_data, _ = create_dataset(client_train_path, batch_size)
    test_data, _ = create_dataset(client_test_path, batch_size)
    clients_train_data.append(train_data)
    clients_test_data.append(test_data)

def create_model(num_classes):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

def client_train(model, train_data, client_num, epochs=1):
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    print(f"Compiling model for client {client_num + 1}")
    for epoch in range(epochs):
        print(f"Client {client_num + 1}, Epoch {epoch + 1}/{epochs}")
        model.fit(train_data, epochs=1, verbose=1)
    return model.get_weights()

def fedwpr_aggregation(local_weights, RR):
    N = len(local_weights)
    aggregated_weights = []

    for layer_weights in zip(*local_weights):
        rr_component = np.zeros_like(layer_weights[0])
        one_minus_rr_component = np.zeros_like(layer_weights[0])

        # Calculate RR component
        for weights in layer_weights:
            rr_component += RR * np.array(weights) / N

        # Calculate (1 - RR) component
        for weights in layer_weights:
            one_minus_rr_component += (1 - RR) * np.array(weights) / N

        # Combine both components
        new_layer_weights = rr_component + one_minus_rr_component
        aggregated_weights.append(new_layer_weights)

    return aggregated_weights

# Simulate federated learning with FedFV and FedWPR
global_model = create_model(num_classes=num_classes)  # Each client has 19 classes
num_rounds = 1
num_epochs = 1  # Set the number of epochs for each client per round
RR = 0.5  # Set the RR value as needed

for round_num in range(num_rounds):
    print(f"Federated Round {round_num + 1}/{num_rounds}")
    local_weights = []
    for client_num, train_data in enumerate(clients_train_data):
        print(f"Training Client {client_num + 1}")
        client_model = create_model(num_classes=num_classes)
        client_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])  # Ensure the model is compiled before setting weights
        client_model.set_weights(global_model.get_weights())
        client_weights = client_train(client_model, train_data, client_num, epochs=num_epochs)
        local_weights.append(client_weights)

    # Apply FedWPR aggregation
    print("Aggregating local models with FedWPR")
    new_global_weights = fedwpr_aggregation(local_weights, RR)
    global_model.set_weights(new_global_weights)

    # Distribute the aggregated model back to clients
    for client_num, train_data in enumerate(clients_train_data):
        client_model.set_weights(new_global_weights)

    print(f"Round {round_num + 1} completed")

# Save the global model
global_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])  # Ensure the global model is compiled
global_model.save('federated_global_model_fedwpr.h5')

def evaluate_model(model, test_data):
    loss, accuracy = model.evaluate(test_data, verbose=0)
    return loss, accuracy

# Evaluate the global model on each client's test data
for client_num, test_data in enumerate(clients_test_data):
    loss, accuracy = evaluate_model(global_model, test_data)
    print(f'Client {client_num + 1} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')
